In [1]:
1+1

2

In [2]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("Speech.txt")

/Users/ashoksaidoredla/Documents/Langchain_LLM/Langchain_venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
text_docs = loader.load()
text_docs

[Document(metadata={'source': 'Speech.txt'}, page_content='"My dear fellow citizens,"\n\nNamaste! Today, as we stand at a crucial moment in our nation\'s journey, I want to talk about the power of self-reliance, innovation, and unity in building a stronger India.\n\nIndia is not just a country; it is an emotion, a rich heritage, and a land of limitless possibilities. Our young minds, our farmers, our entrepreneurs, and our hardworking citizens are shaping a future that the world admires. Today, India is a leader in technology, space research, digital transformation, and manufacturing. We are moving towards an Atmanirbhar Bharat—a self-reliant India that believes in Made in India, Made for the World.\n\nThe world looks at India with hope and trust. Our advancements in AI, healthcare, and infrastructure are setting global benchmarks. From Chandrayaan’s success to digital payments revolution, India is scripting a new history. But true success is when every Indian prospers, when opportunit

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
# os.environ['']

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

# Retreivert and Chain with LLM

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Doredla_Ashok_Sai_Resume.pdf")
docs = loader.load()
docs

[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-03-16T18:14:40-04:00', 'title': 'Doredla_Ashok_Sai_Resume_Data_Analyst', 'author': 'Ashok Sai Doredla', 'moddate': '2025-03-16T18:14:40-04:00', 'source': 'Doredla_Ashok_Sai_Resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='ASHO K SAI DO RE DLA \n              Baltimore, MD, 21229 | 484 -202-0211 | doredlaashoksai@gmail.com  | LinkedI n | GitHub   \n \nPRO FE SSIO NAL SUMMARY \n \n \nResults-driven Data Professional  with around 3 years of experience  in machine learning, Generative AI, and Large Language Models (LLMs) . Proficient \nin Python, T ensorFlow, PyT orch, and NLP, with a strong background in data preprocessing, predictive modeling, and AI -based automation . Developed \ndeep learning models achieving 82% classification accuracy and optimized database performance by 45% . Dedicated to AI-driven innovation , data-driven \ndecision-making, a

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 20)
documents = text_splitter.split_documents(docs)[:6]


[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-03-16T18:14:40-04:00', 'title': 'Doredla_Ashok_Sai_Resume_Data_Analyst', 'author': 'Ashok Sai Doredla', 'moddate': '2025-03-16T18:14:40-04:00', 'source': 'Doredla_Ashok_Sai_Resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='ASHO K SAI DO RE DLA \n              Baltimore, MD, 21229 | 484 -202-0211 | doredlaashoksai@gmail.com  | LinkedI n | GitHub   \n \nPRO FE SSIO NAL SUMMARY \n \n \nResults-driven Data Professional  with around 3 years of experience  in machine learning, Generative AI, and Large Language Models (LLMs) . Proficient \nin Python, T ensorFlow, PyT orch, and NLP, with a strong background in data preprocessing, predictive modeling, and AI -based automation . Developed \ndeep learning models achieving 82% classification accuracy and optimized database performance by 45% . Dedicated to AI-driven innovation , data-driven \ndecision-making, a

In [8]:
documents = text_splitter.split_documents(docs)

In [10]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.from_documents(documents, OllamaEmbeddings())

In [11]:
db

In [13]:
query = "What is the Work Experience"
result = db.similarity_search(query)
result[0].page_content

'• Optimized PostgreSQL databases using indexing, and query caching, improving query efficiency by 45%.  \n• Created interactive Power BI  dashboards to visualize key business metrics, such as KPIs  and sales performanc e, driving actionable insights. \n• Collaborated with cross -functional teams  to present insights and recommendations to leadership, demonstrating strong communication skills.  \n \nHexaware Technologies,  Associate Data Analyst,  Chennai , India                                      Dec 2020 – Oct 2021  \n• Conducted exploratory data analysis  (EDA) using visualizations  and statistical analysis, such as hypothesis testing , to improve model accuracy. \n• Assisted in building E T L pipelines and data automation  workflows, ensuring efficient data integration and transformation.   \n• Identified data inconsistencies and improved preprocessing pipelines , increasing reliability.'

In [16]:
from langchain_community.llms import Ollama

llm = Ollama(model = "llama3.2")
llm

Ollama(model='llama3.2')

In [19]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input} """)

In [20]:
from langchain.chains.combine_documents import create_stuff_documents_chain

documnet_chain = create_stuff_documents_chain(llm, prompt)

In [21]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x114ea47c0>, search_kwargs={})

In [22]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, documnet_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x114ea47c0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context. \nThink step by step before providing a detailed answer. \nI will tip 

In [24]:
response = retrieval_chain.invoke({"input":"What is the Work Experience"})
response['answer']

'Based on the provided context, I can identify that "Work Experience" refers to the professional work done by the individual during their tenure.\n\nThe text highlights various roles and responsibilities held by the individual, including:\n\n* Associate Data Analyst at Hexaware Technologies\n* Conducted data analysis, ETL pipeline development, and data automation workflows\n\nMore specific details about these experiences are mentioned in separate sections of the text, but overall, it appears that the work experience includes a mix of data analysis, software development (specifically using Python and various AI/ML frameworks), collaboration with teams, and data-driven insights generation.'

In [26]:
response = retrieval_chain.invoke({"input":"What is the location for my Masters Education"})
response['answer']

'Based on the provided context, the location for your Master\'s education is:\n\nUniversity of Maryland, Baltimore County (UMBC), Maryland, USA. \n\nAdditionally, it is mentioned that you completed your degree there, as indicated by "Expected May 2025" under the EDUCATION section, implying that the completion date has passed and you have graduated.'